In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import os

from sklearn.model_selection import train_test_split
import sklearn.decomposition as decomposition
from sklearn.manifold import TSNE

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt
import seaborn as sb
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as mpatches

In [2]:
df_train = pd.read_csv("data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

n_class = np.unique(Y).size

X.shape, Y.shape

((87554, 187, 1), (87554,))

In [3]:
seq_len = 187

def get_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(128, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=True,
                   dropout=0.2))
    model.add(LSTM(128, 
                   activation='relu',
                   return_sequences=False, 
                   dropout=0.1))
    
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

def get_simple_model():
    n_class = 5
    model = Sequential()
    model.add(LSTM(32, 
                   input_shape=(seq_len, 1), 
                   activation='relu', 
                   return_sequences=False,
                   dropout=0.2))
    
    model.add(Dense(32, activation='relu'))
    #model.add(Dropout(0.2))
    model.add(Dense(n_class, activation='softmax'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
    
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

In [4]:
model = get_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 187, 128)          66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 32)                4128      
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 165       
Total params: 202,437
Trainable params: 202,437
Non-trainable params: 0
_________________________________________________________________


In [5]:
mini_X = X[:1000, :, :]
mini_Y = Y[:1000]

np.unique(mini_Y, return_counts=True)

(array([0, 1, 2, 3, 4], dtype=int8),
 array([815,  32,  68,  10,  75], dtype=int64))

In [ ]:
logdir = os.path.join("logs", "scalars", str(datetime.now().strftime('%Y%m%d-%H%M%S')))
if not os.path.exists(logdir):
    os.makedirs(logdir)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

model.fit(X[:, :, :], Y[:], 
          epochs=10, 
          verbose=1,
          batch_size=32,
          validation_split=0.1,
          callbacks=[tensorboard_callback])

Train on 78798 samples, validate on 8756 samples
Epoch 1/10
78798/78798 [==============================] - 802s 10ms/sample - loss: 9359519107187246.0000 - accuracy: 0.8276 - val_loss: 0.6797 - val_accuracy: 0.8278
Epoch 2/10
78798/78798 [==============================] - 785s 10ms/sample - loss: 0.6658 - accuracy: 0.8277 - val_loss: 0.6575 - val_accuracy: 0.8278
Epoch 3/10
18048/78798 [=====>........................] - ETA: 12:40 - loss: 0.6620 - accuracy: 0.8261

In [ ]:
model.predict(mini_X)